In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from argparse import ArgumentParser, Namespace
from random import choices
import pytorch_lightning as pl
from typing import Callable, List, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData
from gpu_spatial_graph_pipeline.utils import adata2data
from gpu_spatial_graph_pipeline.data.datamodule import GraphAnnDataModule
from gpu_spatial_graph_pipeline.models.linear_ncem import LinearNCEM

In [3]:
#Mibitof
adata = sq.datasets.mibitof()
#feature_name=adata.obs.keys()[0] #Use for IMC dataset

#specify features to use
feature_names=['Cluster','batch']

def mibitof2data(adata):
    return adata2data(adata,feature_names)


#input of datamodule
num_features=(len(set(adata.obs[feature_names[0]])),len(set(adata.obs[feature_names[1]])))
num_genes=adata.X.shape[1]


In [4]:
dm = GraphAnnDataModule(adata=adata, adata2data_fn=mibitof2data, num_workers = 12, batch_size=40,learning_type='nodewise')

In [5]:
dm.setup()

In [6]:
itr = 4
for batch in dm.train_dataloader():
    print(batch)
    itr -= 1
    if itr<0:
        break

DataBatch(x=[262, 11], edge_index=[2, 240], y=[262, 36], batch=[262], ptr=[4], train_mask=[262], val_mask=[262], test_mask=[262], batch_size=40)
DataBatch(x=[265, 11], edge_index=[2, 240], y=[265, 36], batch=[265], ptr=[4], train_mask=[265], val_mask=[265], test_mask=[265], batch_size=40)
DataBatch(x=[270, 11], edge_index=[2, 240], y=[270, 36], batch=[270], ptr=[4], train_mask=[270], val_mask=[270], test_mask=[270], batch_size=40)
DataBatch(x=[270, 11], edge_index=[2, 240], y=[270, 36], batch=[270], ptr=[4], train_mask=[270], val_mask=[270], test_mask=[270], batch_size=40)
DataBatch(x=[272, 11], edge_index=[2, 240], y=[272, 36], batch=[272], ptr=[4], train_mask=[272], val_mask=[272], test_mask=[272], batch_size=40)


In [7]:
model = LinearNCEM(in_channels=num_features,out_channels=num_genes, model_type='spatial', lr=0.0001,weight_decay=0.000001)

In [10]:
gpu=False
if gpu:
    trainer:pl.Trainer = pl.Trainer(accelerator='gpu',max_epochs=10,log_every_n_steps=10)
else:
    trainer:pl.Trainer = pl.Trainer(accelerator='cpu',max_epochs=10,log_every_n_steps=10)


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/sel/anaconda3/envs/graph/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1764: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [11]:
trainer.fit(model,datamodule=dm)


  | Name        | Type            | Params
------------------------------------------------
0 | model_sigma | LinearSpatial   | 2.7 K 
1 | model_mu    | LinearSpatial   | 2.7 K 
2 | loss_module | GaussianNLLLoss | 0     
------------------------------------------------
5.5 K     Trainable params
0         Non-trainable params
5.5 K     Total params
0.022     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Exception ignored in: <function _releaseLock at 0x7fbcf6fa6940>
Traceback (most recent call last):
  File "/home/sel/anaconda3/envs/graph/lib/python3.8/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


RuntimeError: DataLoader worker (pid(s) 12925, 12937, 12938, 12961, 12973, 12985, 12986, 13009, 13021) exited unexpectedly

In [62]:
trainer.test(model, datamodule=dm)

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           -0.1223994642496109
      test_r2_score        -0.04144535355516912
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_r2_score': -0.04144535355516912, 'test_loss': -0.1223994642496109}]